# wfdiff

The source code for `wfdiff` lives on Github: https://github.com/krischer/wfdiff. If you encounter any problems with it please open an issue or submit a pull request.

## Installation

### Installing Python and Dependencies

`wfdiff` has a couple of dependencies. If you are well versed in Python, then the installation should not prove an issue, otherwise, please follow the advice here and download the [Anaconda Python distribution](http://continuum.io/downloads) for your system. In case you don't know which version to choose, choose Python 3.4. `wfdiff` nonetheless also works with Python 2.7, 3.3, and 3.4.

After downloading and installing Anaconda, update it with

```bash
$ conda update conda
```

Then create a new environment. You don't have to, but using a new environment grants a clean seperation between Python installations.

```bash
$ conda create -n wfdiff python=3.4
```

This will create a new conda environment based on Python 3.4 named `wfdiff`. Activate it with

```bash
$ source activate wfdiff
```

(On windows just with `$ activate wfdiff`). Remember to activate it everytime you want to use `wfdiff`.

Now install ObsPy with

```bash
$ conda install -c obspy obspy
```

and the remaining dependencies with

```bash
$ conda install basemap pandas flake8 pytest nose pyasdf tqdm
```

### Install wfdiff

To install it, best use `pip` (does not yet work):

```bash
$ pip install wfdiff
```

If you want the latest development version, or want to work on the code, you will have to install with the help of `git`.

```bash
$ git clone https://github.com/krischer/wfdiff.git
$ cd wfdiff
$ pip install -v -e .
```

## Tests

To assure the installation is valid and everything works as expected, run the tests with

```bash
$ python -m wfdiff.tests
```

## Running wfdiff

To run it, create a Python file, e.g. `run_wfdiff.py` with the following content:

```python
from wfdiff.wfdiff import WFDiff

# Also see:
#   run_wfdiff_test.py - Examples for using specfema and asdf files as input
#   pyasdf_example.py  - To convert specfem files to asdf files

# A non-existant directory where the output will be stored.
OUTPUT_DIRECTORY = "output"

# Choose which misfits you want and the threshold values.
THRESHOLDS = {
    "rms": 0.25,
    "l1_norm": 0.3}
# A couple more misfits are available.
# These currently take a long time to calculate.
#    "cross_correlation": 0.9,
#    "phase_misfit": 0.1,
#    "envelope_misfit": 0.1}


# Configuration.
c = WFDiff(
    # Low resolution waveform files. UNIX wildcards must be used.
    low_res_seismos="../test_data/NGLL5/*.semd",
    # High resolution waveform files. UNIX wildcards must be used.
    high_res_seismos="../test_data/NGLL7/*.semd",
    # Stations file.
    # Set this to 'None' for asdf files
    station_info="../test_data/STATIONS",
    # Event file
    # Set this to 'None' for asdf files
    event_file="../test_data/CMTSOLUTION",
    # Specify the units of the data and the units the analysis should take
    # place in.
    data_units="displacement", desired_analysis_units="displacement",
    # Periods to test.
    t_min=1, t_max=8, dt=0.5,
    # Data window to take into account in seconds since the first sample.
    starttime=0, endtime=199,
    # Rotate to RTZ (needs 3 channels for each station)
    rotate_RTZ = True,
    # Set to 'True' if specfem filename are in NET.STA.CHA.* format
    # Set to 'False' if specfem filename are in STA.NET.CHA.* format
    new_specfem_name=True,
    # Tags printed on the plots
    trace_tags=trace_tags,
    # asdf dataset tags
    asdf_tags=asdf_tags,
    # Set to 'specfem' if waveform ASCII files are used. 
    # Set to 'asdf' if asdf waveform file
    # All other fileformat should otherwise work just fine.
    wf_format=wf_format)

# Perform the frequency dependent misfit measurements. Results will be stored
# in 'results.json' in the output directory. This file can later be read again
# if necessary to perform the subsequent analysis.
results = c.run(
    misfit_types=list(THRESHOLDS.keys()),
    output_directory=OUTPUT_DIRECTORY,
    # Setting this to True will create A LOT of debug plots which are very useful
    # to tune the algorithm to the problem at hand. For production runs set this to
    # False.
    save_debug_plots=True,
    output_format='pdf')


# This produces all kinds of plots for all components and misfits it encounters.
results.plot_all(
    output_directory=OUTPUT_DIRECTORY,
    thresholds=THRESHOLDS,
    # Event file (for plotting the beachball)
    # You can also use asdf dataset file instead of CMTSOLUTION file
    event_file="../test_data/CMTSOLUTION",
    output_format='pdf')
```

Run it with 

```bash
$ python run_wfdiff.py
```

As these calculations can potentially take a long time, you can also run it with MPI:

```bash
$ mpirun -n 16 python run_wfdiff.py
```

The public interfaces of `wfdiff` can correctly deal with MPI and will parallelize the code if possible. Please make sure to not run anything else in the Python file or be aware of what is actually happening.